In [ ]:
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Create a sample dataset
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])

